In [7]:
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.plot import show
import rasterio.mask
from rasterio import features
# import rasterstats
import numpy as np
import csv
import fiona
from pathlib import Path

In [2]:
city_list = pd.read_csv('centroids.csv').city

### AVERAGE TEMPERATURE

In [10]:
def avg_temp(city):
    city_nospace = city.replace(" ", "_").lower()
    temp_file = city / Path('data') / (city_nospace + '_Summer.tif')
    temp = rasterio.open(temp_file)
    temp_array = temp.read(1)
    return np.nanmean(temp_array)

In [11]:
cities_avg_temp = {}
for city in city_list:
    cities_avg_temp[city] = avg_temp(city)

In [12]:
cities_avg_temp

{'Kinshasa': 35.334088518862714,
 'Kananga': 32.00891289547679,
 'Mbuji-Mayi': 36.85760845577679,
 'Lubumbashi': 30.08157966178147,
 'Kisangani': 35.38010622791592,
 'Bukavu': 30.60558524615587,
 'Goma': 30.993772087777092,
 'Tshikapa': 34.6169967091788,
 'Mwene-Ditu': 32.194247272321405,
 'Gemena': 32.73614193509841,
 'Gbadolite': 35.612178989775835,
 'Matadi': 36.203785944691816,
 'Kikwit': 34.163992372762614,
 'Bunia': 32.40720093952636}

In [13]:
with open('stats/avg_temp.csv', 'w') as f:
    f.write('city,avg\n')
    for city in cities_avg_temp.keys():
        f.write("%s,%s\n"%(city, cities_avg_temp[city]))

### AIR QUALITY

In [18]:
def avg_air(city, year):
    city_nospace = city.replace(" ", "_").lower()
    temp_file = city / Path('data') / (city_nospace + '_air_quality_' + str(year) + '.tif')
    temp = rasterio.open(temp_file)
    temp_array = temp.read(1)
    temp_array = temp_array[temp_array >= 0]
    return np.nanmean(temp_array)

In [19]:
def bad_air(city, threshold = 5):  # threshold for the definition of "bad air"; default is 5 ug/m3
    city_nospace = city.replace(" ", "_").lower()
    air_file = city / Path('data') / (city_nospace + '_air_quality_' + str(year) + '.tif')
    air = rasterio.open(air_file)
    air_array = air.read(1)
    return sum(sum(air_array >= threshold)) / sum(sum(air_array != air.meta['nodata']))

In [20]:
cities_avg_air = {}
for city in city_list:
    cities_avg_air[city] = {}
    for year in range(1998, 2020):
        cities_avg_air[city][year] = avg_air(city, year)

In [21]:
cities_bad_air = {}
for city in city_list:
    cities_bad_air[city] = {}
    for year in range(1998, 2020):
        cities_bad_air[city][year] = bad_air(city)

In [24]:
# cities_avg_air

In [22]:
with open('stats/avg_air_1998_2019.csv', 'w') as f:
    f.write('city,year,avg,pct_bad_air\n')
    for city in cities_avg_air.keys():
        for year in range(1998, 2020):
            f.write("%s,%s,%s,%s\n"%(city, year, cities_avg_air[city][year], cities_bad_air[city][year]))

### LANDSLIDE AVERAGE FREQUENCY

In [23]:
def landslide(city):
    city_nospace = city.replace(" ", "_").lower()
    ls_file = city / Path('data') / (city_nospace + '_landslide.tif')
    ls = rasterio.open(ls_file)
    ls_array = ls.read(1)
    return np.nanmean(ls_array)

In [24]:
cities_ls = {}
for city in city_list:
    try:
        landslide_value = landslide(city)
        cities_ls[city] = landslide_value
    except:
        pass

In [25]:
cities_ls

{'Kinshasa': 0.0007153648,
 'Lubumbashi': 4.2331605e-05,
 'Bukavu': 0.0028261074,
 'Goma': 0.0007660687,
 'Matadi': 0.00036825065}

In [26]:
with open('stats/landslide_avg.csv', 'w') as f:
    f.write('city,avg\n')
    for city in cities_ls.keys():
        f.write("%s,%s\n"%(city, cities_ls[city]))